In [20]:
import pandas as pd

In [21]:
df = pd.read_csv("../../Data/model_0_pre_feat_selection.csv")

In [ ]:

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler



#Create X without wl_time
X = df.drop(['wl_time'], axis=1)

wl_time = df['wl_time']
mean_wl_time = wl_time.mean()
print(f"Mean of wl_time: {mean_wl_time}")
wl_time = wl_time.apply(lambda x: 1 if x > mean_wl_time else 0)

y = wl_time

#import log
log = LogisticRegression(max_iter=1000)

selector = RFECV(estimator=log, cv=5, scoring='f1', verbose=1)
selector.fit(X, y)

print(f"Num of features: {X.shape[1]}")
print(f"Num of features recommended after feature selection: {selector.n_features_}")

Mean of wl_time: 192.52839426430694
Fitting estimator with 109 features.
Fitting estimator with 108 features.
Fitting estimator with 107 features.
Fitting estimator with 106 features.
Fitting estimator with 105 features.
Fitting estimator with 104 features.


In [ ]:
feature_rankings = list(zip(df.columns, selector.ranking_, selector.support_))
feature_rankings_sorted = sorted(feature_rankings, key=lambda x: x[1])
for feature, ranking, support in feature_rankings_sorted:
    print(f"Feature: {feature}, Ranking: {ranking}, Support: {support}")

In [ ]:
# Remove all features that don't have support
X_new = df[[col for col in df.columns if col in X.columns[selector.support_]]]

#Print number of features in X_new
print(f"Num of features in X_new: {X_new.shape[1]}")

#Print if X_new contains wl_time column
if 'wl_time' in X_new.columns:
    print("X_new contains wl_time column")

# Concatenate X_new and y
df_new = pd.concat([X_new, y], axis=1)

In [ ]:
#Print the weights of the model with the corresponding feature name
weights = list(zip(X_new.columns, log.coef_[0]))
weights_sorted = sorted(weights, key=lambda x: x[1])
for feature, weight in weights_sorted:
    print(f"Feature: {feature}, Weight: {weight}")

In [ ]:
df_new.to_csv(f"../../Data/model_0_post_feat_selection.csv", index=False)